# **Tutorial**

**IMPORTANT** : Run this once in the beginning :

We navigate to the directory of the executable files.  
You must end up in the *larvaworld/run* directory

In [1]:
import os
os.chdir('../run')
!pwd

/home/panos/nawrot_larvaworld/larvaworld/run


For all the executable py files various flags are available.  
The ordering of the flags does not matter.  
Help regarding the accepted flags is available via the **-h** flag.

## **Process existing data**


Handling and analysis of existing data can be carried out using the **process.py**.  

This is applicable to both empirical and simulated data.

Let's check the help :

In [2]:
!python process.py -h

usage: process.py [-h] [-fld FOLDERS [FOLDERS ...]]
                  [-suf SUFFIXES [SUFFIXES ...]] [-nam NAMES [NAMES ...]]
                  [-load] [-d_ids DATASET_IDS [DATASET_IDS ...]]
                  [-raw RAW_FOLDERS [RAW_FOLDERS ...]]
                  [-t [MIN_DURATION_IN_SEC]] [-all] [-each] [-hide]
                  [-vid [VIDEO_SPEED]] [-img [{final,overlap,snapshots}]]
                  [-rnd] [-beh] [-trj [TRAJECTORIES]] [-blk] [-head]
                  [-aln {origin,arena,center}] [-dyn {lin,ang}]
                  [-ids AGENT_IDS [AGENT_IDS ...]]
                  [-tkr TICK_RANGE [TICK_RANGE ...]]
                  [-fix FIX_POINTS [FIX_POINTS ...]] [-Nsegs [DRAW_NSEGS]]
                  [-dim ARENA_DIMS [ARENA_DIMS ...]] [-con] [-mid] [-cen]
                  DataGroup_id {reg,init,build,enrich,anal,vis}
                  [{reg,init,build,enrich,anal,vis} ...]

Initialize processing

positional arguments:
  DataGroup_id          The id of the DataGroup to process.

### **Registration**
-------------------

We define a new DataGroup named *TestGroup* by using the action **reg**  
This opens an interactive dialoque prompting the user for appropriate input.  
Initially we are asked to either use an existing configuration or define a new one.  
Then we define the path, subgroups and arena parameters of the DataGroup   

Note : the interactive dialogue only runs in linux terminal. For this tutorial, the DataGroup is already defined.

In [3]:
!python process.py TestGroup reg

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
 ----- Registration of new DataGroup : TestGroup ----- 
 -- Step 1 : DataGroup Configuration
Use an existing configuration?  (default : False) :^C
Traceback (most recent call last):
  File "process.py", line 32, in <module>
    setDataGroup(DataGroup_id)
  File "../lib/stor/datagroup.py", line 200, in setDataGroup
    if get_input("Use an existing configuration? ", itype=bool, default=False):
  File "../lib/stor/datagroup.py", line 14, in get_input
    t = input(f'{message} (default : {default}) :')
KeyboardInterrupt


We initialize the directory structure of the new DataGroup by using the action **init**.  
This is required for any further processing.  
Two basic folders are created under the DataGroup path :  
- **raw** : Any raw files as generated by the tracker. These can be transformed to the larvaworld format by the *build* action, once the relevant build configuration is in place. The files must be added manually.  
- **processed** : Datasets in larvaworld format that can be further analysed. These are either added manually or generated by the *build* action.  

Note : For this tutorial, the *raw* folder of the DataGroup already exists containing some raw recordings (see below)

In [4]:
!python process.py TestGroup init

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In the *raw* folder of the DataGroup there are 3 directories.   
Each contains an empirical 3' recording of a dish of freely-exploring larvae.


In [5]:
!ls ../data/TestGroup/raw

dish_0	dish_1	dish_2


### **Building**
-----------------

We transform the raw recordings to the larvaworld format by the *build* action.
To select the raw data you can use the following flags :
- **-all**  : a single merged dataset will be created from all raw data in the DataGroup raw directory.    
- **-each** : a separate dataset will be created for each folder in the raw directory.  
- **-raw**  : manually define the raw data folders for each dataset to be created.  

Additionally :
- **-t** and a float : demand a minimum track duration in seconds for the included larva tracks.
- **-d_ids** : manually define the unique ids of the new datasets

Here we first create a dataset per dish.  

In [6]:
!python process.py TestGroup build -each

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Initialized dataset dish_0 with new configuration
Initialized dataset dish_1 with new configuration
Initialized dataset dish_2 with new configuration
3 datasets loaded.
Saving data
Dataset dish_0 created with 15 larvae!
Saving data
Dataset dish_1 created with 15 larvae!
Saving data
Dataset dish_2 created with 15 larvae!


In the *processed* folder of the DataGroup there are now 3 directories.  
Each contains a dataset containing larva tracks from the respective dish.  
The datasets are in larvaworld format and can be analysed further.

In [7]:
!ls ../data/TestGroup/processed

dish_0	dish_1	dish_2


Let's have a look at the default directory structure of a larvaworld dataset :
- **data** : The data and configuration of the dataset
    - step_dataset.csv : The step by step data. One row per timestep per larva
    - endpoint_dataset.csv : The endpoint data. One row per larva
    - config_dataset.csv : The dataset configuration. It is loaded into python as a dictionary.
- **aux** : Auxiliary files generated during enrichment and useful for plotting.
- **plots** : The directory to store analysis plots.
- **visuals** : The directory to store dataset visualizations.


In [8]:
!ls ../data/TestGroup/processed/dish_0
!ls ../data/TestGroup/processed/dish_0/data

aux  data  plots  visuals
dataset_conf.csv  endpoint_data.csv  step_data.csv


The step_data and endpoint_data are the main data files for each dataset.  

 - In step_data there exists one row per larva per timestep. Parameter values are stored for each timestep.
 
 - In endpoint_data there exists only a single row per larva. Single values of endpoint parameters are stored for each larva. 
 
 Let's have a look :

In [14]:
import pandas as pd

dir = '../data/TestGroup/processed/dish_0/data'

s = pd.read_csv(f'{dir}/step_data.csv', index_col=['Step', 'AgentID'])
e = pd.read_csv(f'{dir}/endpoint_data.csv', index_col=0)

s.head(30)

head_x    head_y   point2_x  point2_y   point3_x  point3_y  \
Step AgentID                                                                   
1    Larva_1    0.085251 -39.16210   0.354475 -39.14060   0.622776 -39.05170   
     Larva_10        NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_11  -7.057730   3.62680  -6.882520   3.30950  -6.674880   3.00165   
     Larva_12  -4.176850  12.93810  -3.929190  12.78750  -3.697280  12.60710   
     Larva_13  11.288400   3.15309  11.021900   2.93235  10.666700   2.76185   
     Larva_14        NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_15        NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_2         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_3         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_4         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_5    8.533320  13.32350   8.268520  13.57440   8.027330  13.75660   
     Larva_6         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_7         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_8   -6.439580  24.42820  -6.369370  24.11180  -6.273660  23.79380   
     Larva_9   -5.889540  36.01240  -5.611990  35.79780  -5.323700  35.63170   
2    Larva_1    0.137947 -39.29890   0.366075 -39.18370   0.612652 -39.03080   
     Larva_10        NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_11  -6.782610   3.71590  -6.768580   3.39436  -6.642410   3.05190   
     Larva_12  -4.153300  12.95790  -3.929580  12.80260  -3.680500  12.59520   
     Larva_13  11.451400   3.12327  11.166400   2.96144  10.802600   2.85175   
     Larva_14        NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_15        NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_2         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_3         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_4         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_5    8.652960  13.34810   8.366300  13.57540   8.113070  13.75270   
     Larva_6         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_7         NaN       NaN        NaN       NaN        NaN       NaN   
     Larva_8   -6.445340  24.44420  -6.389160  24.13560  -6.295460  23.80230   
     Larva_9   -6.041010  35.98310  -5.723920  35.81050  -5.411920  35.65940   

                point4_x  point4_y  point5_x  point5_y  ...  contour17_y  \
Step AgentID                                            ...                
1    Larva_1    0.887458 -38.91570   1.10511 -38.76170  ...    -38.08630   
     Larva_10        NaN       NaN       NaN       NaN  ...          NaN   
     Larva_11  -6.471380   2.69319  -6.27093   2.40638  ...      2.13080   
     Larva_12  -3.466170  12.38900  -3.25051  12.14260  ...     11.74240   
     Larva_13  10.297300   2.70499   9.94162   2.70131  ...      2.36227   
     Larva_14        NaN       NaN       NaN       NaN  ...          NaN   
     Larva_15        NaN       NaN       NaN       NaN  ...          NaN   
     Larva_2         NaN       NaN       NaN       NaN  ...          NaN   
     Larva_3         NaN       NaN       NaN       NaN  ...          NaN   
     Larva_4         NaN       NaN       NaN       NaN  ...          NaN   
     Larva_5    7.737990  13.93220   7.46466  14.08470  ...     14.17480   
     Larva_6         NaN       NaN       NaN       NaN  ...          NaN   
     Larva_7         NaN       NaN       NaN       NaN  ...          NaN   
     Larva_8   -6.138200  23.49150  -5.94347  23.20980  ...     23.01780   
     Larva_9   -4.995050  35.44760  -4.69156  35.26240  ...     35.49600   
2    Larva_1    0.833811 -38.89120   1.08725 -38.71990  ...    -38.16380   
     Larva_10        NaN       NaN       NaN       NaN  ...          NaN   
     Larva_11

In [15]:
e.head(15)

num_ticks   cum_dur
AgentID                      
Larva_1        2880  180.0000
Larva_10       2858  178.6250
Larva_11       2880  180.0000
Larva_12       2880  180.0000
Larva_13       2851  178.1875
Larva_14       1392   87.0000
Larva_15       1842  115.1250
Larva_2        1857  116.0625
Larva_3        2117  132.3125
Larva_4        1698  106.1250
Larva_5        2851  178.1875
Larva_6        1978  123.6250
Larva_7        2858  178.6250
Larva_8        2880  180.0000
Larva_9        2880  180.0000

Let's create also a merged dataset with the larva tracks longer than 175 seconds from all dishes.  

In [16]:
!python process.py TestGroup build -all -t 175

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Initialized dataset merged with new configuration
1 datasets loaded.
Saving data
Dataset merged created with 28 larvae!


Let's check the endpoint file to make sure the individual dishes were merged in a single dataset containing all the larvae

In [7]:
!ls ../data/TestGroup/processed
!ls ../data/TestGroup/processed/merged

dish_0	dish_1	dish_2	merged
aux  data  plots  visuals


In [17]:
dir = '../data/TestGroup/processed/merged/data'

s = pd.read_csv(f'{dir}/step_data.csv', index_col=['Step', 'AgentID'])
e = pd.read_csv(f'{dir}/endpoint_data.csv', index_col=0)

e.head(30)

num_ticks   cum_dur
AgentID                      
Larva_1        2880  180.0000
Larva_10       2859  178.6875
Larva_11       2880  180.0000
Larva_12       2880  180.0000
Larva_13       2880  180.0000
Larva_14       2880  180.0000
Larva_15       2880  180.0000
Larva_16       2880  180.0000
Larva_17       2880  180.0000
Larva_18       2880  180.0000
Larva_19       2878  179.8750
Larva_2        2851  178.1875
Larva_20       2880  180.0000
Larva_21       2880  180.0000
Larva_22       2880  180.0000
Larva_23       2880  180.0000
Larva_24       2880  180.0000
Larva_25       2880  180.0000
Larva_26       2878  179.8750
Larva_27       2848  178.0000
Larva_28       2848  178.0000
Larva_3        2858  178.6250
Larva_4        2880  180.0000
Larva_5        2880  180.0000
Larva_6        2858  178.6250
Larva_7        2880  180.0000
Larva_8        2880  180.0000
Larva_9        2851  178.1875

### **Selection**
-----------------

To select specific datasets for processing the following flags are available :
- **-fld** : Folders under the DataGroup proceesed directory where to search for datasets
- **-nam** : Names of the datasets to select. All provided folders will be searched.
- **-suf** : Suffixes to attach to the names

### **Enrichment**
-----------------

These datasets only contain the primary parameters already contained in the raw files (mostly xy coordinates of midline and contour points).  
We can enrich the datasets with derived parameters and annotate behavioral epochs.  
The enrichment configuration is contained in the DataGroup configuration as defined above.  

Let's apply enrichment to both the separate-dish datasets...

In [18]:
!python process.py TestGroup enrich -nam dish_0 dish_1 dish_2

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Resumed dataset dish_0 with existing configuration
Loading data from /home/panos/nawrot_larvaworld/larvaworld/lib/stor/../../data/TestGroup/processed/dish_0/data/step_data.csv
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
Resumed dataset dish_1 with existing configuration
Loading data from /home/panos/nawrot_larvaworld/larvaworld/lib/stor/../../data/TestGroup/processed/dish_1/data/step_data.csv
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
Resumed dataset dish_2 with existing configuration
Loading data from /home/panos/nawrot_larvaworld/larvaworld/lib/stor/../../data/TestGroup/processed/dish_2/data/step_data.csv
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
3 datasets loaded.
Rows excluded according to collision_flag.
Applying filter to all spatial parameter

... and the merged dataset.  

In [20]:
!python process.py TestGroup enrich -nam merged

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Resumed dataset merged with existing configuration
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
1 datasets loaded.
Rows excluded according to collision_flag.
Applying filter to all spatial parameters
All parameters filtered
Computing body length
All lengths computed.
Computing centroid from 22 contourpoints
Centroid coordinates computed.
Computing front and rear orientations
All orientations computed
Computing 5 angles
All angles computed
Computing bending angle as the sum of the first 5 front angles
All bends computed
All orientations unwrapped
Computing angular velocities and accelerations for 3 angular parameters
All angular parameters computed
LR biases computed
Completed minimal angular analysis.
Detecting chunks-on-condition for 28 agents
Computing chunk metrics for 28 agents
Chunk metrics computed
All chunks-on-condition detected
Detecting chunks

Let's have a look at the enriched merged dataset.    
The *aux* folder contains files generated by the enrichment.

In [21]:
!ls ../data/TestGroup/processed
!ls ../data/TestGroup/processed/merged

dish_0	dish_1	dish_2	merged
aux  data  plots  visuals


In [23]:
dir = '../data/TestGroup/processed/merged/data'

s = pd.read_csv(f'{dir}/step_data.csv', index_col=['Step', 'AgentID'])
e = pd.read_csv(f'{dir}/endpoint_data.csv', index_col=0)

e.head(30)

num_ticks   cum_dur    length  bend_mean  bend_vel_mean   bend_std  \
AgentID                                                                        
Larva_1        2750  171.8750  3.328786  -3.219831       0.938406  34.073038   
Larva_10       2686  167.8750  4.344017  -1.250413       0.027627  36.657377   
Larva_11       2754  172.1250  3.932295  -5.573283       1.196286  35.531910   
Larva_12       2717  169.8125  3.296080  -2.166636      -2.218993  31.303940   
Larva_13       2721  170.0625  3.620774   3.608704       0.416342  24.286841   
Larva_14       2828  176.7500  3.236055  -4.196527      -1.033195  24.174024   
Larva_15       2757  172.3125  3.629784 -10.267861      -0.330068  36.098023   
Larva_16       2861  178.8125  3.302567  -1.117128      -0.304726  18.140541   
Larva_17       2757  172.3125  3.790382  -1.361413      -0.734441  36.091107   
Larva_18       2769  173.0625  2.907257  -4.745626      -0.132937  15.703750   
Larva_19       2536  158.5000  3.617597  -3.752882       1.415147  32.696483   
Larva_2        2753  172.0625  3.579977  -0.479888      -0.382293  21.357262   
Larva_20       2731  170.6875  4.004451  -1.757180       1.073981  39.151121   
Larva_21       2771  173.1875  3.913139   4.507163      -5.393803  44.584342   
Larva_22       2845  177.8125  3.965995  -6.177359      -1.721658  26.649291   
Larva_23       2808  175.5000  3.767806  -5.553462       0.563221  19.149218   
Larva_24       2809  175.5625  3.884421  -0.134742      -0.512637  33.157483   
Larva_25       2817  176.0625  3.309184  -8.253025      -1.762595  22.349472   
Larva_26       2648  165.5000  3.593047  -4.942744       0.893547  41.081393   
Larva_27       2767  172.9375  3.692777   5.241519       1.274957  25.276238   
Larva_28       2800  175.0000  3.198865  -5.343489       1.775311  21.724660   
Larva_3        2710  169.3750  4.245945  -2.423466      -2.112521  36.797099   
Larva_4        2816  176.0000  3.478548  -8.054995      -2.009625  33.319230   
Larva_5        2742  171.3750  4.003442  -1.919141       0.169638  28.450478   
Larva_6        2739  171.1875  4.129944   6.005458       2.676329  31.491270   
Larva_7        2795  174.6875  3.873953  -1.968543       0.717240  35.513787   
Larva_8        2847  177.9375  3.345376  -5.247664      -0.363292  18.855929   
Larva_9        2745  171.5625  3.921886   4.893890      -2.095032  29.581615   

          bend_vel_std  num_Lturns  cum_Lturn_dur  Lturn_dur_mean  ...  \
AgentID                                                            ...   
Larva_1     106.915061       219.0        68.7500        0.313927  ...   
Larva_10    132.145146       224.0        69.8750        0.311942  ...   
Larva_11    106.841263       235.0        61.5000        0.261702  ...   
Larva_12    111.552317       202.0        65.8125        0.325804  ...   
Larva_13     72.633659       231.0        65.8750        0.285173  ...   
Larva_14     65.881968       237.0        74.1250        0.312764  ...   
Larva_15    113.242764       219.0        60.8125        0.277683  ...   
Larva_16     62.595814       262.0        72.8750        0.278149  ...   
Larva_17    154.046887       226.0        71.0000        0.314159  ...   
Larva_18    151.913191       267.0        67.2500        0.251873  ...   
Larva_19    122.293919       216.0        65.3125        0.302373  ...   
Larva_2      86.717940       248.0        70.0625        0.282510  ...   
Larva_20    114.055042       192.0        74.4375        0.387695  ...   
Larva_21    165.958296       226.0        72.7500        0.321903  ...   
Larva_22     82.475855       234.0        69.3750        0.296474  ...   
Larva_23     50.571115       242.0        63.8750        0.263946  ...   
Larva_24     85.078403       250.0        67.5625        0.270250  ...   
Larva_25     79.765784       258.0        68.2500        0.264535  ...   
Larva_26    144.685529       210.0        63.3750        0.301786  ...   
Larva_27     98.256062       235.0        79.1875      

In [12]:
!ls ../data/TestGroup/processed/merged/aux
!ls ../data/TestGroup/processed/merged/aux/par_distros

dispersion  par_distros  par_during_stride
Lturn_front_orientation_unwrapped.csv
Rturn_front_orientation_unwrapped.csv
bend.csv
bend_acc.csv
bend_at_stride_start.csv
bend_at_stride_stop.csv
bend_vel.csv
centroid_dst_at_stride_start.csv
centroid_dst_at_stride_stop.csv
centroid_x_at_stride_start.csv
centroid_x_at_stride_stop.csv
centroid_y_at_stride_start.csv
centroid_y_at_stride_stop.csv
front_orientation_acc.csv
front_orientation_unwrapped_at_Lturn_start.csv
front_orientation_unwrapped_at_Lturn_stop.csv
front_orientation_unwrapped_at_Rturn_start.csv
front_orientation_unwrapped_at_Rturn_stop.csv
front_orientation_unwrapped_at_stride_start.csv
front_orientation_unwrapped_at_stride_stop.csv
front_orientation_vel.csv
pause_dur.csv
rear_orientation_acc.csv
rear_orientation_unwrapped_at_stride_start.csv
rear_orientation_unwrapped_at_stride_stop.csv
rear_orientation_vel.csv
scaled_vel_at_stride_start.csv
scaled_vel_at_stride_stop.csv
stride_bend.csv
stride_centroid_dst.csv
stride_centroid_x.c

### **Analysis**
-----------------

Let's now analyse the enriched merged dataset using the **anal** action.  

In [27]:
!python process.py TestGroup anal -nam merged

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Resumed dataset merged with existing configuration
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
1 datasets loaded.
Bend correction during strides for merged fitted as : db=-0.82*b + 2.05
Plot saved as stride_bend_change.pdf
Plot saved as stride_orient_change.pdf
Plot saved as scaled_dispersion_0-40_2.pdf
Plot saved as scaled_dispersion_0-40_1.pdf
Plot saved as dispersion_0-40_2.pdf
Plot saved as dispersion_0-40_1.pdf
Plot saved as angular_pars_3.pdf
Plot saved as angular_pars_7.pdf
Plot saved as angular_pars_5.pdf
No handles with labels found to put in legend.
No handles with labels found to put in legend.
Plot saved as stridesNpauses_pdf_default_0.pdf
Fits saved as bout_fits_default.csv.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
Plot saved as stridesNpauses_cdf_default_0.pdf
Fits saved as bout_fits_de

Plot files are generated in the respective folder.  

In [30]:
!ls ../data/TestGroup/processed/merged/plots/comparative

angular_pars_3.pdf  angular_pars_7.pdf	dispersion  final.pdf	  stride
angular_pars_5.pdf  bouts		endpoint    interference  turn


Let's have a look at some of the plots. 

Some of the endpoint parameters.

In [35]:
!evince ../data/TestGroup/processed/merged/plots/comparative/endpoint/endpoint_params_minimal.pdf

In [16]:
!evince ../data/TestGroup/processed/merged/plots/comparative/endpoint/crawl_pars.pdf

The distribution of angular parameters merged for all the larvae.

In [17]:
!evince ../data/TestGroup/processed/merged/plots/comparative/angular_pars.pdf

If we include multiple datasets, the analysis will generate comparative plots.  
Here we analyse comparatively all dishes.  

In [18]:
!python process.py TestGroup anal -nam dish_0 dish_1 dish_2

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loaded dataset dish_0 with existing configuration
Loading data from /home/panos/Desktop/flyworld/lib/stor/../../data/TestGroup/processed/dish_0/data/step_data.csv
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
Loaded dataset dish_1 with existing configuration
Loading data from /home/panos/Desktop/flyworld/lib/stor/../../data/TestGroup/processed/dish_1/data/step_data.csv
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
Loaded dataset dish_2 with existing configuration
Loading data from /home/panos/Desktop/flyworld/lib/stor/../../data/TestGroup/processed/dish_2/data/step_data.csv
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
3 datasets loaded.
Plot saved as endpoint_params_minimal.pdf
Tests saved as endpoint_ttest.csv.
Plot saved as endpoint_params_limited.pdf
Tes

When multiple datasets are compared, the plots are stored in the parent DataGroup plot folder.

In [19]:
!ls ../data/TestGroup/plots

angular_pars.pdf		   interference_orientation_agent_idx_0.pdf
bend_correction.pdf		   interference_orientation_agent_idx_1.pdf
bout_fits			   interference_spinelength.pdf
crawl_pars.pdf			   interference_spinelength_agent_idx_0.pdf
dispersion			   interference_spinelength_agent_idx_1.pdf
endpoint_params_full.pdf	   orientation_over_strides.pdf
endpoint_params_limited.pdf	   stridesNpauses_cdf_broad.pdf
endpoint_params_minimal.pdf	   stridesNpauses_cdf_default.pdf
endpoint_ttest.csv		   stridesNpauses_cdf_restricted.pdf
final.pdf			   stridesNpauses_pdf_broad.pdf
interference_bend.pdf		   stridesNpauses_pdf_default.pdf
interference_bend_agent_idx_0.pdf  stridesNpauses_pdf_restricted.pdf
interference_bend_agent_idx_1.pdf  turn_duration.pdf
interference_orientation.pdf	   turns.pdf


Again let's have a look at some comparative plots. 

The distribution of crawl-runs and crawl-pauses.

In [20]:
!evince ../data/TestGroup/plots/stridesNpauses_cdf_broad.pdf

The dispersion from origin for the first 40 seconds (scal to the larva-length).

In [21]:
!evince ../data/TestGroup/plots/dispersion/scaled_dispersion_0-40.pdf

The linear velocity and angular front and rear orientation velocities phasically locked to the peristaltic stride cycle.

In [22]:
!evince ../data/TestGroup/plots/interference_orientation.pdf

### **Visualization**
-----------------

We can visualize a dataset using the **vis** action.  

**IMPORTANT** : The same flags can be used for visualizing simulations when running *exp_run.py*

#### **Video and image options**

Let's generate a video of dish 1 (dish_1) using the **-vid** flag

In [3]:
!python process.py TestGroup vis -nam dish_1 -vid

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Resumed dataset dish_1 with existing configuration
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
1 datasets loaded.
Screen opened
  9% (280 of 2880) |##                   | Elapsed Time: 0:00:17 ETA:   0:02:41^C

Traceback (most recent call last):
  File "process.py", line 42, in <module>
    visualize_datasets(DataGroup_id, **data_kwargs, vis_kwargs = {**vis_kwargs, **replay_kwargs})
  File "../lib/stor/managing.py", line 129, in visualize_datasets
    d.visualize(save_to=save_to, save_as=n, **vis_kwargs)
  File "../lib/stor/larva_dataset.py", line 781, in visualize
    replay_env.run()
  File "../lib/model/envs/_larvaworld.py", line 396, in run
    self.render(background_motion=self.background_motion[:, i])
  File "../lib/model/envs/_larvaworld.py", line 335, in render
    self._screen.render()
  File "../lib/aux/rendering.py", line 137, in render
    

Or a final image of the dish (**-img final**) or just (**-img**)

In [4]:
!python process.py TestGroup vis -nam dish_1 -img

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Resumed dataset dish_1 with existing configuration
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
1 datasets loaded.
Screen opened
100% (2880 of 2880) |####################| Elapsed Time: 0:00:00 ETA:  00:00:00
Visualization complete
Screen closed


Or snapshots at regular intervals (**-img snapshots**)

In [5]:
!python process.py TestGroup vis -nam dish_1 -img snapshots

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Resumed dataset dish_1 with existing configuration
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
1 datasets loaded.
Screen opened
Screen closed
Screen closed
 70% (2022 of 2880) |##############      | Elapsed Time: 0:00:18 ETA:   0:00:18^C

Traceback (most recent call last):
  File "process.py", line 42, in <module>
    visualize_datasets(DataGroup_id, **data_kwargs, vis_kwargs = {**vis_kwargs, **replay_kwargs})
  File "../lib/stor/managing.py", line 129, in visualize_datasets
    d.visualize(save_to=save_to, save_as=n, **vis_kwargs)
  File "../lib/stor/larva_dataset.py", line 781, in visualize
    replay_env.run()
  File "../lib/model/envs/_larvaworld.py", line 401, in run
    self.step()
  File "../lib/model/envs/_larvaworld.py", line 848, in step
    self.active_larva_schedule.step()
  File "/home/panos/nawrot_larvaworld/my_venv/lib/python3.6/site-pac

We can collapse the whole video on a single image by overlapping all subsequent images (**-img overlap**)  

See below in the replay section for a nice application of this.

#### **Trajectories**

We can visualize the full trajectories (**-trj**)

In [6]:
!python process.py TestGroup vis -nam dish_1 -img -trj

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Resumed dataset dish_1 with existing configuration
Step data loaded according to types dictionary
Data loaded successfully from stored csv files.
1 datasets loaded.
Screen opened
100% (2880 of 2880) |####################| Elapsed Time: 0:00:00 ETA:  00:00:00
Visualization complete
Screen closed


Or show trajectories of 10 seconds (**-trj 10**). Here a video of the merged dataset.

In [ ]:
!python process.py TestGroup vis -nam merged -vid -trj 10

#### **Colors**

 We can color each larva with a unique random color (**-rnd**)

In [ ]:
!python process.py TestGroup  vis -nam merged -vid -trj 10 -rnd

We can also color the larvae according to the annotated behavior **(-beh)**:
- green : Peristaltic stride
- light green flash : End of peristaltic stride
- red : Pause of crawling
- gold : Feeding motion (only applicable to simulated data
- black (or default color) : Not annotated

In [ ]:
!python process.py TestGroup -nam dish_1 vis -vid -trj 10 -beh

We can also visualize on a black background **(-blk)**

In [ ]:
!python process.py TestGroup -nam dish_1 vis -vid -blk

### Replay
-----------------

#### **Specific larvae**

We can select specific larvae by using the **-ids** flag plus the unique ids of the larvae or their indexes in the dataset.  
Here we select 3 specific larvae (**-ids 2 6 12**) of the merged dataset.

In [ ]:
!python process.py TestGroup vis -nam merged -vid -ids 2 6 12

#### **Dynamic trajectories**

We can dynamically color trajectories according to :  
- linear velocity (**-dyn lin**)
- angular velocity (**-dyn ang**)

Here we illustrate linear velocity. We also color larvae according to behavioral annotation (see *Visualization*)

In [ ]:
!python process.py TestGroup vis -nam merged -vid -ids 2 3 4 -trj -dyn lin -beh

And a final image of a single trajectory illustrating the angular (front orientation) velocity 

In [ ]:
!python process.py TestGroup vis -nam merged -img -ids 11 -trj -dyn ang

#### **Trajectory transposition**

We can transpose/align the larva tracks using the **-aln** flag.  
The larva tracks are recomputed before visualization.  
The following modes are available :  
- **origin** : Each track originates from the exact center of the arena. Relative track relations are lost.    
- **center** : Each track is centered around the arena center. Relative track relations are lost.  
- **arena**  : All tracks are centered to the arena. Relative track relations are preserved.  

Let's make all trajectories start from the same origin at the cenetr of the screen (**-aln origin**) to visualize dispersion over time.

In [ ]:
!python process.py TestGroup vis -nam merged -vid -trj -aln origin

We can centralize the trajectory to the center of the screen using the alignment flag (**-aln center**)

In [ ]:
!python process.py TestGroup vis -nam merged -img -ids 11 -trj -dyn ang -aln center

#### **Arena dimensions**

The arena dimensions are set via the - **-dim** flag.   
- If a single float follows the flag, a circular dish of given radius.  
- If two floats follow the flag, a rectangular arena of given dimensions. 

We can now use a rectangular arena 40x50 mm to zoom in (**-dim 0.04 0.05**) on our previous single trajectory.

**IMPORTANT** : If the trajectory exceeds the arena dimensions an error will be generated.

In [ ]:
!python process.py TestGroup vis -nam merged -img -ids 11 -trj -dyn lin -aln center -dim 0.04 0.05 

Or watch the respective video

In [ ]:
!python process.py TestGroup vis -nam merged -vid -ids 11 -trj -dyn ang -aln center -dim 0.04 0.05 -beh

#### **Larva fixation**

We can also fixate a midline point of a single larva to the center of the screen.  
This allows us to have the larva constantly under focus.  
A background is generated that moves accordingly.  

Let's fixate the larva's 8th midline point (starting from the 0th being the head).  **(-fix 8)**  
Of course we can now use an even smaller arena. Here a dish of radius 0.008 mm **(-dim 0.008)**

In [ ]:
!python process.py TestGroup vis -nam merged -vid -ids 11 -fix 8 -dim 0.008

We can fixate the whole segment above or below the fixated point by typing -1 (above segment) or 1 (below segment) after its index.  
Now the larva is always facing up (with 1) or down (with -1)

Let's fixate the segment above **(-fix 8 -1)**

In [ ]:
!python process.py TestGroup vis -nam merged -vid -ids 11 -fix 8 -1 -dim 0.008

#### **Midline & contour**

Let's try the same without the midline **(-mid)**

In [ ]:
!python process.py TestGroup vis -nam merged -vid -ids 11 -fix 8 -1 -dim 0.008 -mid

 Or without the contour **(-con)**.

In [ ]:
!python process.py TestGroup vis -nam merged -vid -ids 11 -fix 8 -1 -dim 0.008 -con

Now we can collapse the whole video on a single image by overlapping all subsequent images **(-img overlap)**

In [ ]:
!python process.py TestGroup vis -nam merged -img overlap -ids 11 -fix 8 -1 -dim 0.008 -con

#### **Time slice**

We can also visualize a specific time slice of the dataset.
We have to define a range in timesteps (easy to compute using the framerate of the tracking. For this dataset it is 16 fps)

Let's visualize the first dish  **(-idx 1)** for the range 320 to 640 timesteps (20-40 seconds) **(-tkr 320 640)**

In [1]:
!python process.py TestGroup vis -nam dish_1 -vid -tkr 320 640

python: can't open file 'process.py': [Errno 2] No such file or directory


## **Simulations**

We always need to define the type of experiment to run.   
We can optionally set a number of parameters. If not set the default values for the type of experiment will be used :
- **-id** : Simulation id
- **-N** : Number of larvae.
- **-t** : Simulation duration in minutes

### **Single runs**

Single simulations of behavioral experiments can be run using the **exp_run.py**.  

We can optionally run the relevant analysis on the simulated data once the simulation is finished by using the **-a** flag.  
We can use all flags descibed in *Visualization*.  

Let's check the help :

In [3]:
!python exp_run.py -h

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
usage: exp_run.py [-h] [-a] [-id SIM_ID] [-N NAGENTS] [-t SIM_TIME] [-dt DT]
                  [-Box2D] [-age [HOURS_AS_LARVA]] [-deb_f [DEB_BASE_F]]
                  [-starv_h STARVATION_HOURS [STARVATION_HOURS ...]] [-hide]
                  [-vid [VIDEO_SPEED]] [-img [{final,overlap,snapshots}]]
                  [-rnd] [-beh] [-trj [TRAJECTORIES]] [-blk] [-head]
                  {focus,dish,dispersion,chemorbit,chemotax,odor_pref,feed_grid,feed_patchy,feed_scatter,growth,growth_2x,maze,imitation,reorientation}

Run given experiments

positional arguments:
  {focus,dish,dispersion,chemorbit,chemotax,odor_pref,feed_grid,feed_patchy,feed_scatter,growth,growth_2x,maze,imitation,reorientation}
                        The experiment type

optional arguments:
  -h, --help            show this help message and exit
  -a, --analysis        Whether to run analysis
  -id SIM_ID, --sim_id SIM_ID
            

The collected simulation dataset is stored at the **larvaworld/data/simulation/single_runs** folder under the respective experiment type and simulation id.

Let's run a **dish** experiment. Larvae freely explore a plain dish. No odors or food are present. 

In [4]:
!python exp_run.py dish -N 20 -t 3.0 -vid

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
3.0
Initialized dataset dish_205 with new configuration
Initialized simulation dish_205!
Screen opened
  3% (67 of 1800) |                      | Elapsed Time: 0:00:06 ETA:   0:02:53Screen closed
Screen closed
100% (1800 of 1800) |####################| Elapsed Time: 0:00:07 Time:  0:00:07
Simulation not completed!


#### **Chemotaxis**

There are 3 available experiments involving chemotaxis :
- **chemotax** : Larvae start on the left of the arena facing right. An odor source exists at the other side.
- **chemorbit** : Larvae start on the vincinity of an odor source.
- **odor_pref** : Larvae start on the middle of the arena. Odor sources of different valence exist on either side.

In [ ]:
!python exp_run.py chemotax -N 20 -t 3.0 -vid

Let's run a **chemorbit** experiment on the background and apply analysis :

In [ ]:
!python exp_run.py chemorbit -N 20 -t 3.0 -img -trj -rnd -a -id my_chemorbit

Let's locate the simulated dataset :

In [ ]:
!ls ../data/simulation/single_runs/chemorbit
!ls ../data/simulation/single_runs/chemorbit/my_chemorbit/plots

Let's see the odorscape ...

In [ ]:
!eog ../data/simulation/single_runs/chemorbit/my_chemorbit/plots/Random_odor_ID_odorscape.png

... the final image ...

In [ ]:
!eog ../data/simulation/single_runs/chemorbit/my_chemorbit/visuals/my_chemorbit_0.png

... and the sensed odor concentration across time.  
Red denotes a single larva. Grey denotes the median and quantiles.

In [ ]:
!evince ../data/simulation/single_runs/chemorbit/my_chemorbit/plots/odor_concentration.pdf

Let's now run an **odor_pref** experiment.  

The analysis immediately computes and prints the Preference Index

In [ ]:
!python exp_run.py odor_pref -N 20 -t 3.0 -img -a -id my_odor_pref

#### **Feeding**

There are 4 available experiments involving feeding :
- **feed_grid** : Food sources are aranged in a rectangular grid.
- **feed_scatter** : Circular food sources are randomly placed in the arena.
- **feed_patchy** : Circular food patches are placed in a circle.
- **growth** : The larvae start as first instars and grow in size as they eat.

Let's run a **feed_grid** experiment.  Note the colors illustrating behavior.  

In [ ]:
!python exp_run.py feed_grid -N 5 -t 3.0 -vid -beh

Here is a **feed_patchy** experiment, saving the final image

In [ ]:
!python exp_run.py feed_patchy -N 20 -t 10.0 -img -trj -rnd -id my_feed_patchy

Let's see the odorscape and the final image :

In [ ]:
!eog ../data/simulation/single_runs/feed_patchy/my_feed_patchy/plots/Random_odor_ID_odorscape.png
!eog ../data/simulation/single_runs/feed_patchy/my_feed_patchy/visuals/my_feed_patchy_0.png

Here is a **growth** experiment over 1 day (1440 minutes).  
We perform the analysis to see the length and mass over time.  

In [ ]:
!python exp_run.py growth -N 1 -t 1440.0 -id my_growth -a

Let's see the graphs

In [ ]:
!evince ../data/simulation/single_runs/growth/my_growth/plots/length_timeplot.pdf
!evince ../data/simulation/single_runs/growth/my_growth/plots/mass_timeplot.pdf

If you are patient enough run the simulation in video mode and watch the larva grow over time :

In [ ]:
!python exp_run.py growth -vid -N 1 -t 1440.0 -beh

#### **Comparison to empirical data**

Let's run a **dispersion** experiment. Larvae freely explore a big arena starting from the exact same point at the center of the screen. No odors or food are present. 

Let's just produce a final image of the full trajectories in random colors.

In [ ]:
!python exp_run.py dispersion -N 20 -t 2.0 -img -trj -rnd

The **dispersion** experiment affords extensive analysis including comparison to the existing **reference** empirical dataset.  
During the analysis the data are enriched as described above in **Enrichment** and analysed as described in **Analysis**

Now let's run the experiment again with the analysis.  
We will also provide a simulation id to easily locate the results afterwards.

In [ ]:
!python exp_run.py dispersion -N 5 -t 2.0 -img -trj -rnd -a -id my_sim

Let's locate the simulated dataset :

In [ ]:
!ls ../data/simulation/single_runs/dispersion

Let's have a look at the image we produced :

In [ ]:
!ls ../data/simulation/single_runs/dispersion/my_sim/visuals
!eog ../data/simulation/single_runs/dispersion/my_sim/visuals/my_sim_0.png

When generating comparative plots empirical data are always in blue and simualted data in red.  
Let's have a look at some of the plots.  

Let's start from simple graphs of linear and angular velocity of individual larvae.

In [ ]:
!ls ../data/simulation/single_runs/dispersion/my_sim/plots/marked_epochs

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/marked_epochs/Larva_0_marked_strides_full.pdf
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/marked_epochs/Larva_0_marked_turns_min_angle_0_full.pdf

And now the comparative plots

In [ ]:
!ls ../data/simulation/single_runs/dispersion/my_sim/plots/comparative

Some endpoint parameters. The circles show statistical significance. 

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/comparative/endpoint_params_minimal.pdf

The distributions of angular parameters across all larvae.

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/comparative/angular_pars.pdf

The log-log distribution of crawl-run and crawl-pause durations along with the best fitting distribution among power-law, exponential and log-normal

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/comparative/stridesNpauses_cdf_broad.pdf

The average linear and angular velocity curve during a stride.

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/comparative/interference_orientation.pdf

The distribution of front and rear orientation change over individual strides

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/comparative/orientation_over_strides.pdf

The amplitude of turns

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/comparative/turns.pdf

The dispersion for the first 40''

In [ ]:
!evince ../data/simulation/single_runs/dispersion/my_sim/plots/comparative/dispersion/dispersion_0-40.pdf

### **Batch runs**

Batch runs can be run using the **batch_run.py**.   

**IMPORTANT** : If a batch run with the same id already exists, it will be resumed expanding the space search with the new one defined.

Let's check the help :

In [ ]:
!python batch_run.py -h

#### **Space search**

We can run multiple single runs of an experiment using different combinations of parameters (configurations).  
The available flags are :
- **-par** : The parameters that participate in the space search.
- **-rng** (floats): The ranges for the parameters. Two floats for each parameter.
- **-Ngrd** (int(s)): The grid dimension to parse the range of each parameter. If a single int is provided it will be used for all parameter ranges.

For the **odor_pref** batch run the parameters are the left and right odor valences.  

Let's run a batch run searching the space of odor valences from -100 to 100 creating a grid of 5x5.  
This will run 25 experiments. Each will have a different combination of left-right odor valences.

In [ ]:
!python batch_run.py odor_pref -N 20 -t 2.0 -rng -100 100 -Ngrd 5 -id my_odor_pref_batch

Let's see the outcome of the batch run.  
A heatmap is generated showing the preference index (PI) for every combination of valences.

In [ ]:
!evince ../data/simulation/batch_runs/odor_pref/my_odor_pref_batch/PI_heatmap.pdf

#### **Optimization**

We can continue the above space search to optimize a specific endpoint fit parameter.  
If the threshold set for the fit parameter is not reached after the initial space search a new generation of configurations will be generated by mutating the best performining ones.  

The available flags are :
- **-Nmax** (int): The maximum number of simulations to run throughout the batch run.
- **-Nbst** (int): The number of best parameter combinations to mutate at the next generation.

For example the **chemorbit** batch run optimizes configurations for achieving the smallest dispersion away from the odor source.  
The default space consists of 25 configurations, therefore 25 runs.  
Let's run a maximum of 50 single runs, selecting the best 5 at each generation.

In [ ]:
!python batch_run.py chemorbit -N 5 -t 1.0 -Nmax 50 -Nbst 5 -id my_chemorbit_batch

Let's see the results.
First column is the index of the run.  
Next columns are the configuration of parameters.  
Last column is the fit parameter.  
The lines are sorted in ascending order of the fit parameter (in the case of minimization), therefore the first row displays the best configuration.

In [ ]:
!cat ../data/simulation/batch_runs/chemorbit/my_chemorbit_batch/results.csv | column -t -s,